In [4]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import leaguepedia_parser as lolp
import dataclasses

pd.set_option('display.min_rows', None)
plt.style.use("dark_background")

In [5]:
df = pd.read_csv("pick_bans_updated.csv", index_col="index")

C:\Users\chiuy\AppData\Local\Temp/ipykernel_14524/1563938094.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("pick_bans_updated.csv", index_col="index")


In [3]:
dir(lolp)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'get_all_team_assets',
 'get_game_details',
 'get_games',
 'get_long_team_name_from_trigram',
 'get_regions',
 'get_team_logo',
 'get_team_thumbnail',
 'get_tournaments',
 'parsers',
 'site',
 'transmuters']

In [4]:
lolp.get_tournaments()

[LeaguepediaTournament(name='Vietnam Esports Championship Season 1', start=None, end=None, region='Vietnam', league='VIRESA', leagueShort=None, rulebook=None, tournamentLevel='Primary', isQualifier=True, isPlayoffs=True, isOfficial=True, overviewPage='Vietnam Esports Championship/Season 1'),
 LeaguepediaTournament(name='VCS CL 2023 Group Stage', start=None, end=None, region='Vietnam', league='VCSCL', leagueShort=None, rulebook=None, tournamentLevel='Primary', isQualifier=True, isPlayoffs=True, isOfficial=True, overviewPage='VCS CL/2023 Season/Group Stage'),
 LeaguepediaTournament(name='OGN LoL Invitational 2012', start='2012-01-13', end='2012-01-20', region='Korea', league='OGN', leagueShort=None, rulebook=None, tournamentLevel='Primary', isQualifier=True, isPlayoffs=True, isOfficial=True, overviewPage='OnGameNet LoL Invitational'),
 LeaguepediaTournament(name='MLG 2012 Fall', start='2012-11-02', end='2012-11-04', region='International', league='MLG 2012 Fall', leagueShort=None, rulebo

In [5]:
for a in lolp.get_tournaments():
    print(a.name)
    print(a.overviewPage)

Vietnam Esports Championship Season 1
Vietnam Esports Championship/Season 1
VCS CL 2023 Group Stage
VCS CL/2023 Season/Group Stage
OGN LoL Invitational 2012
OnGameNet LoL Invitational
MLG 2012 Fall
2012 MLG Pro Circuit/Fall/Championship
LAT Regional Finals Season 3
Season 3 Latin America Regional Finals
IWCT 2013
Gamescom 2013/International Wildcard Tournament
VCS A 2013 Winter Qualifiers
VCS A/2013 Season/Winter Qualifiers
VCS A 2014 Spring Promotion
VCS A/2014 Season/Spring Promotion
VCS A 2014 Summer Promotion
VCS A/2014 Season/Summer Promotion
Brazil Regional Finals 2014
CBLOL/2014 Season/Regional Finals
IWCT 2014
2014 Season International Wild Card Tournament
LAC 2014 Colombia
Riot Latin America Cup 2014/Colombia
VCS A 2015 Spring Qualifiers
VCS A/2015 Season/Spring/Qualifiers
LAN 2015 Opening Cup
Latin America Cup 2015/LAN/Opening Cup/Regular Season
LAN 2015 Opening Cup Playoffs
Latin America Cup 2015/LAN/Opening Cup/Playoffs
LAC 2015 Opening Grand Final
Latin America Cup 2015/Op

In [161]:
lolp.get_games("Intel Arabian Cup 2020/United Arab Emirates/Split 1")

[]

In [57]:
for tournament in lolp.get_tournaments():
    if lolp.get_games(tournament.name) != []:
        print(tournament.name)
        print(lolp.get_games(tournament.name))
        print("------------------------------------------------------")

Intel Arabian Cup 2020
[LolGame(sources=EmptyDataclass(), duration=1524, start='2020-10-01T15:29:00+00:00', creation=None, type=None, queue_id=None, patch='10.20', gameVersion=None, winner='RED', teams=LolGameTeams(BLUE=LolGameTeam(bans=[223, 777, 84, 201, 235], players=[LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=54, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=120, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(p

Battle of the Atlantic 2013
[LolGame(sources=EmptyDataclass(), duration=1581, start='2013-12-14T17:00:00+00:00', creation=None, type=None, queue_id=None, patch=None, gameVersion=None, winner='BLUE', teams=LolGameTeams(BLUE=LolGameTeam(bans=[36, 1, 121], players=[LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=58, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=60, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primaryRune

Demacia Cup 2019
[LolGame(sources=EmptyDataclass(), duration=0, start='2019-12-21T03:10:00+00:00', creation=None, type=None, queue_id=None, patch='9.23', gameVersion=None, winner='BLUE', teams=LolGameTeams(BLUE=LolGameTeam(bans=[2, 80, 60, 58, 41], players=[LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=84, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=421, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primaryRuneTree

Demacia Cup 2020
[LolGame(sources=EmptyDataclass(), duration=1888, start='2020-12-21T03:20:00+00:00', creation=None, type=None, queue_id=None, patch='10.25b', gameVersion=None, winner='BLUE', teams=LolGameTeams(BLUE=LolGameTeam(bans=[429, 127, 101, 3, 164], players=[LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=14, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=104, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primar

Demacia Cup 2021
[LolGame(sources=EmptyDataclass(), duration=1596, start='2021-12-17T04:21:00+00:00', creation=None, type=None, queue_id=None, patch='11.24', gameVersion=None, winner='BLUE', teams=LolGameTeams(BLUE=LolGameTeam(bans=[136, 234, 114, 89, 497], players=[LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=164, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=64, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primar

Demacia Cup 2022
[LolGame(sources=EmptyDataclass(), duration=1419, start='2022-12-14T06:05:00+00:00', creation=None, type=None, queue_id=None, patch='12.23', gameVersion=None, winner='BLUE', teams=LolGameTeams(BLUE=LolGameTeam(bans=[0, 0, 0, 0, 0], players=[LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=266, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=234, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primaryRuneTre

DreamHack Summer 2012
[LolGame(sources=EmptyDataclass(), duration=1987, start='2012-06-17T08:00:00+00:00', creation=None, type=None, queue_id=None, patch=None, gameVersion=None, winner='BLUE', teams=LolGameTeams(BLUE=LolGameTeam(bans=[4, 38, 56], players=[LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=39, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=57, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primaryRuneTreeId=

World Cyber Arena 2015
[LolGame(sources=EmptyDataclass(), duration=2092, start='2015-12-17T09:00:00+00:00', creation=None, type=None, queue_id=None, patch=None, gameVersion=None, winner='RED', teams=LolGameTeams(BLUE=LolGameTeam(bans=[238, 13, 127], players=[LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=41, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=60, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primaryRuneTree

In [104]:
details = lolp.get_games("Intel Arabian Cup 2020")
test_dict = dataclasses.asdict(details[0])
test_dict

{'sources': {},
 'duration': 1524,
 'start': '2020-10-01T15:29:00+00:00',
 'creation': None,
 'type': None,
 'queue_id': None,
 'patch': '10.20',
 'gameVersion': None,
 'winner': 'RED',
 'teams': {'BLUE': {'bans': [223, 777, 84, 201, 235],
   'players': [{'primaryRuneTreeId': None,
     'secondaryRuneTreeId': None,
     'championId': 54,
     'id': None,
     'inGameName': None,
     'profileIconId': None,
     'role': None,
     'sources': {},
     'runes': [],
     'summonerSpells': [],
     'endOfGameStats': None,
     'snapshots': [],
     'itemsEvents': [],
     'wardsEvents': [],
     'skillsLevelUpEvents': [],
     'largeMonstersKills': [],
     'levelUpEvents': [],
     'spellsUses': [],
     'specialKills': []},
    {'primaryRuneTreeId': None,
     'secondaryRuneTreeId': None,
     'championId': 120,
     'id': None,
     'inGameName': None,
     'profileIconId': None,
     'role': None,
     'sources': {},
     'runes': [],
     'summonerSpells': [],
     'endOfGameStats': No

In [118]:
print(test_dict["patch"])
print(test_dict["winner"])
print(test_dict["teams"]["BLUE"]["bans"])
for i in range(5):
    print(test_dict["teams"]["BLUE"]["players"][i]["championId"])
print(test_dict["teams"]["RED"]["bans"])
for i in range(5):
    print(test_dict["teams"]["RED"]["players"][i]["championId"])

10.20
RED
[223, 777, 84, 201, 235]
54
120
245
360
412
[98, 8, 58, 42, 61]
39
876
236
51
25


In [9]:
def list_banpicks(tournament):
    details = lolp.get_games(tournament)
    for i in range(len(details)):
        cur_dict = dataclasses.asdict(details[i])
        print(cur_dict["patch"])
        print(cur_dict["winner"])
        print(cur_dict["teams"]["BLUE"]["bans"])
        for i in range(5):
            print(cur_dict["teams"]["BLUE"]["players"][i]["championId"])
        print(cur_dict["teams"]["RED"]["bans"])
        for i in range(5):
            print(cur_dict["teams"]["RED"]["players"][i]["championId"])
        print("---------------------------------------------------")

In [11]:
list_banpicks("2023 Season World Championship/Main Event")

13.19
BLUE
[115, 518, 429, 85, 53]
126
113
61
22
888
[51, 78, 268, 59, 150]
897
64
134
523
526
---------------------------------------------------
13.19
BLUE
[427, 429, 268, 68, 103]
24
163
897
498
12
[61, 7, 57, 111, 526]
887
59
134
81
497
---------------------------------------------------
13.19
BLUE
[134, 429, 57, 68, 888]
897
254
103
145
12
[78, 497, 61, 163, 7]
24
64
518
498
875
---------------------------------------------------
13.19
RED
[429, 163, 85, 518, 58]
86
254
69
498
497
[122, 61, 57, 268, 2]
516
59
103
145
12
---------------------------------------------------
13.19
BLUE
[51, 59, 64, 254, 68]
58
57
61
119
888
[518, 429, 897, 24, 555]
126
526
268
110
12
---------------------------------------------------
13.19
BLUE
[518, 498, 526, 58, 897]
68
78
268
145
89
[57, 61, 254, 111, 53]
516
163
126
429
12
---------------------------------------------------
13.19
BLUE
[78, 429, 64, 12, 234]
24
59
268
498
497
[61, 68, 57, 134, 518]
58
163
126
81
526
-------------------------------

In [12]:
count=0
for t in lolp.get_tournaments():
    if len(lolp.get_games(t.name)) != 0:
        count+=1

print(count)

KeyboardInterrupt: 

In [162]:
count=0
for t in lolp.get_tournaments():
    if len(lolp.get_games(t.overviewPage)) != 0:
        count+=1

print(count)

744


In [163]:
count=0
for t in lolp.get_tournaments():
    if len(lolp.get_games(t.name)) != 0:
        print(t.name)

Intel Arabian Cup 2020
Battle of the Atlantic 2013
Demacia Cup 2019
Demacia Cup 2020
Demacia Cup 2021
Demacia Cup 2022
DreamHack Summer 2012
World Cyber Arena 2015


In [14]:
t_with_data = []
for t in lolp.get_tournaments():
    if len(lolp.get_games(t.overviewPage)) != 0:
        t_with_data.append(t.overviewPage)

In [14]:
t_with_data

['OnGameNet LoL Invitational',
 '2012 MLG Pro Circuit/Fall/Championship',
 'Season 3 Latin America Regional Finals',
 'Gamescom 2013/International Wildcard Tournament',
 'CBLOL/2014 Season/Regional Finals',
 '2014 Season International Wild Card Tournament',
 'Riot Latin America Cup 2014/Colombia',
 'Latin America Cup 2015/LAN/Opening Cup/Regular Season',
 'Latin America Cup 2015/LAN/Opening Cup/Playoffs',
 'Latin America Cup 2015/Opening Cup/Grand Final',
 'Latin America Cup 2015/LAN/Closing Cup/Promotion',
 'Latin America Cup 2015/LAN/Closing Cup/Regular Season',
 'Latin America Cup 2015/LAN/Closing Cup/Playoffs',
 'Latin America Cup/LAN/2016 Season/Opening Cup/Promotion',
 'Latin America Cup 2015/Closing Cup/Grand Final',
 'LMS/2015 Season/Regional Finals',
 '2015 International Wildcard Tournament/Turkey',
 '2015 International Wildcard Tournament/Chile',
 'Latin America Cup/LAN/2016 Season/Opening Cup/Regular Season',
 'Latin America Cup/LAN/2016 Season/Opening Cup/Playoffs',
 'Latin

In [177]:
list_banpicks('Season 3 World Championship')

None
RED
[30, 60, 4]
58
59
79
81
412
[61, 37, 28]
98
254
238
42
9
---------------------------------------------------
None
BLUE
[105, 85, 98]
60
64
103
67
143
[61, 51, 238]
54
254
50
42
412
---------------------------------------------------
None
RED
[83, 61, 58]
68
60
105
42
37
[238, 98, 412]
85
266
103
51
143
---------------------------------------------------
None
BLUE
[37, 61, 98]
154
60
103
67
143
[85, 4, 238]
58
59
105
42
412
---------------------------------------------------
None
BLUE
[254, 412, 98]
58
64
61
42
143
[238, 103, 105]
85
59
79
51
267
---------------------------------------------------
None
BLUE
[38, 22, 103]
75
254
61
42
143
[98, 238, 4]
83
60
127
51
37
---------------------------------------------------
None
BLUE
[59, 64, 412]
58
266
103
42
37
[98, 238, 61]
154
56
131
81
9
---------------------------------------------------
None
BLUE
[85, 42, 143]
27
59
13
51
412
[238, 64, 67]
98
254
103
81
37
---------------------------------------------------
None
RED
[38, 103, 

In [200]:
dataclasses.asdict(lolp.get_games('Season 3 World Championship')[0])["teams"]["RED"]["bans"]

[61, 37, 28]

In [12]:
def store_banpicks(tournament):

    tournament_name = []
    date = []
    duration = []
    
    game_id = []
    champion_id = []
    patch = []
    side = []
    ban_pick = []
    win_lose = []
    details = lolp.get_games(tournament)
    for i in range(len(details)):
        cur_dict = dataclasses.asdict(details[i])
        
        # Blue side bans
        for j in range(len(cur_dict["teams"]["BLUE"]["bans"])):
            game_id.append(i)
            tournament_name.append(tournament)
            date.append(cur_dict["start"][:10])
            duration.append(cur_dict["duration"])
            patch.append(cur_dict["patch"])
            
            champion_id.append(cur_dict["teams"]["BLUE"]["bans"][j])
            side.append("BLUE")
            ban_pick.append("BAN")
            if cur_dict["winner"] == "BLUE":
                win_lose.append("WIN")
            else:
                win_lose.append("LOSE")
                
        # Red side bans
        for k in range(len(cur_dict["teams"]["RED"]["bans"])):
            game_id.append(i)
            tournament_name.append(tournament)
            date.append(cur_dict["start"][:10])
            duration.append(cur_dict["duration"])
            patch.append(cur_dict["patch"])
            
            champion_id.append(cur_dict["teams"]["RED"]["bans"][k])
            side.append("RED")
            ban_pick.append("BAN")
            if cur_dict["winner"] == "RED":
                win_lose.append("WIN")
            else:
                win_lose.append("LOSE")
                
        # Blue side picks
        for l in range(5):
            game_id.append(i)
            tournament_name.append(tournament)
            date.append(cur_dict["start"][:10])
            duration.append(cur_dict["duration"])
            patch.append(cur_dict["patch"])
            
            champion_id.append(cur_dict["teams"]["BLUE"]["players"][l]["championId"])
            side.append("BLUE")
            ban_pick.append("PICK")
            if cur_dict["winner"] == "BLUE":
                win_lose.append("WIN")
            else:
                win_lose.append("LOSE")

        # Red side picks
        for m in range(5):
            game_id.append(i)
            tournament_name.append(tournament)
            date.append(cur_dict["start"][:10])
            duration.append(cur_dict["duration"])
            patch.append(cur_dict["patch"])
            
            champion_id.append(cur_dict["teams"]["RED"]["players"][m]["championId"])
            side.append("RED")
            ban_pick.append("PICK")
            if cur_dict["winner"] == "RED":
                win_lose.append("WIN")
            else:
                win_lose.append("LOSE")
        
    df = pd.DataFrame({"champion_id": champion_id, "patch": patch, "side": side, 
                        "ban_pick": ban_pick, "win_lose": win_lose, "tournament": tournament_name, 
                        "game_id": game_id, "date": date, "duration": duration})
        
    return df

In [26]:
df_worlds_s3 = store_banpicks('Season 3 World Championship')

In [27]:
df_worlds_s3

,champion_id,patch,side,ban_pick,win_lose,tournament,game_id,date,duration
0,30,None,BLUE,BAN,LOSE,Season 3 World Championship,0,2013-09-15,2221
1,60,None,BLUE,BAN,LOSE,Season 3 World Championship,0,2013-09-15,2221
2,4,None,BLUE,BAN,LOSE,Season 3 World Championship,0,2013-09-15,2221
3,61,None,RED,BAN,WIN,Season 3 World Championship,0,2013-09-15,2221
4,37,None,RED,BAN,WIN,Season 3 World Championship,0,2013-09-15,2221
5,28,None,RED,BAN,WIN,Season 3 World Championship,0,2013-09-15,2221
6,58,None,BLUE,PICK,LOSE,Season 3 World Championship,0,2013-09-15,2221
7,59,None,BLUE,PICK,LOSE,Season 3 World Championship,0,2013-09-15,2221
8,79,None,BLUE,PICK,LOSE,Season 3 World Championship,0,2013-09-15,2221
9,81,None,BLUE,PICK,LOSE,Season 3 World Championship,0,2013-09-15,2221


In [36]:
df = pd.DataFrame()
for t in t_with_data:
    cur_df = store_banpicks(t)
    df = df.append(cur_df)
df.reset_index(inplace=True)
df.drop("index", axis=1, inplace=True)

In [10]:
# can switch to show updated version
version = "13.11.1"
version_2 = "13.23.1"
champion_list = pd.read_json(f"https://ddragon.leagueoflegends.com/cdn/{version_2}/data/en_US/champion.json")

champion_list.drop(["type", "format", "version"], axis=1, inplace=True)
champion_list["id"] = ""

for champ in champion_list.index:
    champion_list["id"][champ] = int(champion_list.loc[champ, "data"]["key"])

champion_list.drop("data", axis=1, inplace=True)
champion_list = champion_list.reset_index().rename(columns={"index": "Champion"})
champion_list.to_csv("Champ_IDs.csv")

In [32]:
df.patch.unique()

array(['3.1', '4.13', '4.12', '5.2', '5.3', '5.4', '5.5', '5.9', '5.1',
       '5.11', '5.12', '5.13', '6.1', '6.2', '6.3', '6.4', '6.5', '6.6',
       '6.11', '6.12', '6.13', '6.15', '6.21', '10.2', '10.21', '10.22',
       '5.15', '4.17', '5.6', '5.14', '3.9', '8.1', '8.2', '8.3', '8.4',
       '1.0.0.148', '1.0.0.132', '1.0.0.133', '1.0.0.134', '8.13', '8.15',
       '9.22', '5.7', '6.7', '3.14', '4.6', '4.7', '4.21', '5.21', '5.24',
       '6.8', '7.1', '7.2', '7.3', '7.4', '7.5', '7.11', '7.12', '7.13',
       '7.14', '7.16', '9.1', '9.2', '9.3', '9.4', '9.5', '9.6', '12.1',
       '12.2', '12.3', '12.4', '12.5', '12.11', '12.12', '12.13', '12.14',
       '13.1', '13.3', '13.4', '13.5', '8.5', '8.11', '8.12', '8.14',
       '9.11', '9.12', '9.13', '9.14', '9.15', '9.16', '10.1', '10.4',
       '10.5', '10.7', '10.8', '10.11', '10.12', '10.13', '10.14',
       '10.15', '10.16', '11.1', '11.2', '11.3', '11.4', '11.5', '11.6',
       '11.11', '11.12', '11.13', '11.14', '11.15', '5.8'

In [9]:
df[df["patch"].isnull()==True].groupby(
    ["tournament"])["game_id"].count().reset_index()

,tournament,game_id


In [430]:
# I tried to find out what patch each tournament was played on for ones with missing patch info.
# Most were part of ongoing seasons that spanned multiple patches while the rest did not have available 
# info. For these cases, I have decided to automatically fill in patch info using the date of the game.

14

In [43]:
patch_notes

,index,Date,Season,Patch,New_Champs,Note
0,0,2023-0,Season thirteen,13.11,NaN,DRX skins. Rell Mid-Scope Update. Ivern Mid-Sc...
1,1,2023-0,Season thirteen,13.10,NaN,Midseason Itemization Update. Item Adjustments...
2,2,2023-0,Season thirteen,13.9,NaN,Inkshadow skins. Neeko Mid-Scope Update. Vario...
3,3,2023-0,Season thirteen,13.8,NaN,Chaos and Order skins. Various balance changes.
4,4,2023-0,Season thirteen,13.7,NaN,Cats Versus Dogs skins. Various balance changes.
5,5,2023-0,Season thirteen,13.6,Milio,Faerie Court skins. Various balance changes.
6,6,2023-0,Season thirteen,13.5,NaN,Forsaken Hymn skins. Yuumi Gameplay Update. Va...
7,7,2023-0,Season thirteen,13.4,NaN,2023 Astronauts skins. Various balance changes.
8,8,2023-0,Season thirteen,13.3,NaN,Art & Sustainability Update: Ahri. Comprehensi...
9,9,2023-0,Season thirteen,13.1b,NaN,Grievous Wounds Changes. Fighter Item Changes....


In [37]:
patch_notes = pd.read_csv("lol_patch_notes.csv")
# for i in range(185):
#     patch_notes.Date[i] = patch_notes.Date[i][:-4]
# for i in range(185, 321):
#     patch_notes.Date[i] = patch_notes.Date[i][:-7]
# for i in range(321):
#     patch_notes.Date[i] = patch_notes.Date[i].replace('\n', ' ').strip()
# patch_notes.Date[257] = "June 17 2012"

# for converting str to datetime, pd.to_datetime converts to timestamp obj, which is faster 
# (and better than converting to dt.datetime obj)
# for i in range(321):
#     patch_notes.Date[i] = dt.strptime(patch_notes.Date[i], "%B %d %Y").date()
    
# patch_notes.Date = pd.to_datetime(patch_notes.Date, format="%B %d %Y")

In [38]:
df.date = pd.to_datetime(df.date)

In [39]:
# Info on the patch of tournaments that I found online.
update_patch = pd.DataFrame({"tournament": 
                              ['2012 MLG Pro Circuit/Fall/Championship',
                                '2014 Season Garena Regional Finals',
                                '2014 Season International Wild Card Tournament',
                                '2015 GPL Summer Playoffs',
                                'Battle of the Atlantic 2013',
                                'Gamescom 2013/International Wildcard Tournament',
                                'Latin America Cup 2015/LAN/Closing Cup/Promotion',
                                'Season 3 China Regional Finals',
                                'Season 3 Korea Regional Finals',
                                'Season 3 Latin America Regional Finals',
                                'Season 3 World Championship'], 
                              "patch": 
                              ['1.0.0.148',
                                '4.13',
                                '4.13',
                                '5.14',
                                '3.14',
                                '3.1',
                                '5.12',
                                '3.9',
                                '3.9',
                                '3.9',
                                '3.11']})
update_patch.to_csv("leaguepedia_some_missing_patch_info.csv")

In [22]:
df3 = pd.DataFrame(df.groupby("patch")["game_id"].count()).reset_index()
for i in range(len(df3)):
    df3.loc[i, "patch"] = str(df3.loc[i, "patch"]) 

In [23]:
pd.set_option('display.max_rows', None)
df.head(600)

,champion_id,patch,side,ban_pick,win_lose,tournament,game_id,date,duration
index,,,,,,,,,
0,98,3.1,BLUE,BAN,WIN,Gamescom 2013/International Wildcard Tournament,0,2013-08-20,1900
1,29,3.1,BLUE,BAN,WIN,Gamescom 2013/International Wildcard Tournament,0,2013-08-20,1900
2,59,3.1,BLUE,BAN,WIN,Gamescom 2013/International Wildcard Tournament,0,2013-08-20,1900
3,98,3.1,RED,BAN,LOSE,Gamescom 2013/International Wildcard Tournament,0,2013-08-20,1900
4,29,3.1,RED,BAN,LOSE,Gamescom 2013/International Wildcard Tournament,0,2013-08-20,1900
5,59,3.1,RED,BAN,LOSE,Gamescom 2013/International Wildcard Tournament,0,2013-08-20,1900
6,85,3.1,BLUE,PICK,WIN,Gamescom 2013/International Wildcard Tournament,0,2013-08-20,1900
7,20,3.1,BLUE,PICK,WIN,Gamescom 2013/International Wildcard Tournament,0,2013-08-20,1900
8,238,3.1,BLUE,PICK,WIN,Gamescom 2013/International Wildcard Tournament,0,2013-08-20,1900


In [40]:
pd.DataFrame(df.groupby("patch")["game_id"].count()).reset_index()

,patch,game_id
0,5.1,576
1,5.11,506
2,5.12,642
3,5.13,654
4,5.14,288
5,5.15,310
6,5.2,350
7,5.24,112
8,5.3,396
9,5.4,272


In [41]:
# update main df patch info with info found online.
df2 = df.copy()
for i in df2[df2["patch"].isnull()==True].index:
    for j in range(len(update_patch)):
        if df2.loc[i, "tournament"] == update_patch.loc[j, "tournament"]:
            df2.loc[i, "patch"] = update_patch.loc[j, "patch"]

In [48]:
# update main df patch info by comparing game dates with patch dates (not necessarily accurate).

# took 34 minutes to execute
for i in df2[df2["patch"].isnull()==True].index:
    for j in range(len(patch_notes)):
        if df2.date[i] >= patch_notes.Date[j]:
            df2.loc[i, "patch"] = patch_notes.loc[j, "Patch"]
            break

In [79]:
# drop rows with missing champion_id info (==0)
df2.drop(df2[df2["champion_id"]==0].index, axis=0, inplace=True)
df2.reset_index(inplace=True)
df2.drop(["index"], axis=1, inplace=True)

ValueError: cannot insert level_0, already exists

In [84]:
df2.to_csv("pick_bans_updated.csv")

In [15]:
# Update the final pick_bans list
df_update = pd.DataFrame()
for t in t_with_data:
    if t not in df["tournament"].unique():
        cur_df = store_banpicks(t)
        df_update = pd.concat([df_update, cur_df])
df_update.reset_index(inplace=True)
df_update.drop("index", axis=1, inplace=True)

In [37]:
# if champion_id == 0 for BAN, it means the team did not ban a champion for that slot.
df_update[df_update["champion_id"]==0]
df_update.drop(df_update[df_update["champion_id"]==0].index, axis=0, inplace=True)
df_update.reset_index(inplace=True)
df_update.drop(["index"], axis=1, inplace=True)
df_update.to_csv("pick_bans_updated_2023nov26.csv")

In [44]:
t_with_data

['OnGameNet LoL Invitational',
 '2012 MLG Pro Circuit/Fall/Championship',
 'Season 3 Latin America Regional Finals',
 'Gamescom 2013/International Wildcard Tournament',
 'CBLOL/2014 Season/Regional Finals',
 '2014 Season International Wild Card Tournament',
 'Riot Latin America Cup 2014/Colombia',
 'Latin America Cup 2015/LAN/Opening Cup/Regular Season',
 'Latin America Cup 2015/LAN/Opening Cup/Playoffs',
 'Latin America Cup 2015/Opening Cup/Grand Final',
 'Latin America Cup 2015/LAN/Closing Cup/Promotion',
 'Latin America Cup 2015/LAN/Closing Cup/Regular Season',
 'Latin America Cup 2015/LAN/Closing Cup/Playoffs',
 'Latin America Cup/LAN/2016 Season/Opening Cup/Promotion',
 'Latin America Cup 2015/Closing Cup/Grand Final',
 'LMS/2015 Season/Regional Finals',
 '2015 International Wildcard Tournament/Turkey',
 '2015 International Wildcard Tournament/Chile',
 'Latin America Cup/LAN/2016 Season/Opening Cup/Regular Season',
 'Latin America Cup/LAN/2016 Season/Opening Cup/Playoffs',
 'Latin